In [ ]:
# Add project root to path
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Project imports
from trust_bayesian_agent_comparison.analysis.sensitivity import SensitivityAnalysisManager
from trust_bayesian_agent_comparison.config import *

# Import partner strategies (these will be moved to partners module)
# For now, assume they're available from the old notebook

## Setup Sensitivity Analysis Manager

The manager handles:
- Automatic timestamped file storage
- Loading existing results (no re-computation)
- Organized directory structure by date

In [ ]:
# Initialize the manager
sensitivity_manager = SensitivityAnalysisManager()

# Define partner configurations
# Format: (name, factory_function, threshold_direction)
partner_configs = [
    ("TitForTatCoop", lambda: TitForTatCooperatePartner(), "up"),
    ("TitForTatDefect", lambda: TitForTatDefectPartner(), "down"),
    ("AlwaysDefect", lambda: AlwaysDefectPartner(), "down"),
    ("AlwaysCollaborate", lambda: AlwaysCollaboratePartner(), "up"),
    ("SingleCycleCheater", lambda: SingleCyclePartner(cooperate_rounds=30), "down"),
    ("StrategicCheater", lambda: StrategicCheaterPartner(t_threshold=2.5), "down"),
    ("Random", lambda: RandomPartner(), "down"),
]

## Run Sensitivity Analysis

Set `overwrite=False` to load existing results from today's folder.
Set `overwrite=True` to force re-computation.

In [ ]:
# Run analysis for all partners
# This will:
# 1. Create results/sensitivity/YYYY-MM-DD/ folder
# 2. Save each partner's results as CSV
# 3. Load existing results if overwrite=False

results = sensitivity_manager.run_multiple(
    partner_configs=partner_configs,
    overwrite=False,  # Set to True to force re-run
    # You can override default grids here:
    # eta_grid=np.linspace(0.0, 1.0, 10),
    # seeds=(42, 43, 44, 45, 46)  # More seeds for extra robustness
)

print("\nResults loaded for:")
for name in results.keys():
    print(f"  - {name}: {len(results[name])} simulations")

## Visualize Results

Now you can use the standard plotting functions on the loaded results.

In [ ]:
# Example: Plot E_p sensitivity for one partner
df = results["TitForTatCoop"]

# Aggregate by parameter (average over seeds)
aggregated = df.groupby("loss_aversion").agg({
    "E_p_last": ["mean", "std"],
    "seed": "count"
}).reset_index()

aggregated.columns = ['loss_aversion', 'E_p_mean', 'E_p_std', 'n']
aggregated['E_p_sem'] = aggregated['E_p_std'] / np.sqrt(aggregated['n'])

# Plot
plt.figure(figsize=(10, 6))
plt.errorbar(aggregated['loss_aversion'], aggregated['E_p_mean'], 
             yerr=aggregated['E_p_sem'], marker='o', capsize=5)
plt.axhline(y=2/3, color='red', linestyle='--', label='Decision Threshold')
plt.xlabel('Loss Aversion (λ)')
plt.ylabel('Final E[p]')
plt.title('Sensitivity to Loss Aversion - TitForTatCoop')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## Benefits of This Approach

1. ✅ **Automatic organization**: Results saved in dated folders
2. ✅ **No accidental overwrites**: Old results preserved
3. ✅ **Fast iteration**: Load existing results with `overwrite=False`
4. ✅ **Clean notebooks**: Logic moved to modules
5. ✅ **Reproducible**: Timestamped results
6. ✅ **Scalable**: Easy to add new partners